### User Input

In [ ]:
mfccs_json_path = "../mfccs_cnn_humpbackwhale_walrus_bowheadwhale_fin_finbackwhale_killerwhale.json"

accuracy_loss_image = "/Users/seantedesco/Documents/marine-mammal-call-classification/acc_loss_metrics_2.png"

confusion_mat_image = "/Users/seantedesco/Documents/marine-mammal-call-classification/finalCFcnn.png"

### Imports

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np
import warnings

### Load MFCC Dataset

In [ ]:
from cnn_comparison_lib import load_cnn_json

X, y, L = load_cnn_json(mfccs_json_path)
print(f"mapping the marine mammals: {L}")

In [ ]:
# Check class counts and that labels are numerical order
np.unique(y, return_counts=True)

### Create Training, Validation, and Test Sets

In [ ]:
# create train, validation and test sets
from cnn_comparison_lib import prepare_datasets

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(X, y, 0.8, 0.3) # test size, vailidation size
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

### Build the 3-Layer Models

In [ ]:
from cnn_comparison_lib import create_layered_cnn

L3_model_list = []
for i in range(10):
    output_shape = i*100 + 32
    model = create_layered_cnn(3, input_shape, output_shape)
    L3_model_list.append(model)

### Get 3-Layer Trainable Parameter Count

In [ ]:
from cnn_comparison_lib import count_trainable_parameters

L3_parameter_count = []
for model in L3_model_list:
    count = count_trainable_parameters(model)
    L3_parameter_count.append(count)

print(L3_parameter_count)

### Train the 3-Layer Models

In [ ]:
from cnn_comparison_lib import plot_history

L3_history = []
for model in L3_model_list:
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=16, epochs=5)
    L3_history.append(history)

### Evaluate the 3-Layer Models

In [ ]:
# evaluate the cnn on the test set
L3_accuracy = []
for model in L3_model_list:
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    L3_accuracy.append(test_accuracy)

### Plot the 3-Layer Models

In [ ]:
x = L3_parameter_count
y = L3_accuracy
plt.plot(x, y, 'o', color='black')
plt.xlabel("Number of Parameters")
plt.ylabel("Model Accuracy")